<a href="https://colab.research.google.com/github/5dongdong/PY_AI/blob/master/%EA%B2%B0%EA%B3%BC%EA%B0%92%20acc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score
import time

df = pd.read_csv("C:/Users/bitcamp/Desktop/Medical_noshow3.csv")  

X = df[['Gender_F','Gender_M', 'Diabetes','Hipertension', 'Scholarship', 'SMS_received',
        'Handicap_0','Handicap_1','Handicap_2','Handicap_3','Handicap_4', 'Num_App_Missed', 'Age', 'AwaitingTime']]
y = df["No-show"] 

x_train, x_test, y_train,y_test = train_test_split(
    X, y, train_size=0.8, shuffle=True, random_state=42
)
     # 변수 설정
n_splits = 5    # 보통 홀수로 들어감
random_state = 42
scaler = MinMaxScaler()

kfold = KFold(n_splits=n_splits, shuffle=True, 
              random_state=random_state)


# Scaler
scaler.fit(x_train)                 
x_train = scaler.transform(x_train)   # train 은 fit, transform 모두 해줘야 함
x_test = scaler.transform(x_test) \


from sklearn.ensemble import BaggingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier


# 2. 모델
from sklearn.model_selection import GridSearchCV

# 3. 훈련
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

start_time = time.time()
# 모델 초기화
cat_model = CatBoostClassifier()
lgbm_model = LGBMClassifier()
xgb_model = XGBClassifier()
rf_model = RandomForestClassifier()

# 보팅 분류기 생성
voting_model = VotingClassifier(
estimators=[('cat', cat_model), ('lgbm', lgbm_model), ('xgb', xgb_model)],
    voting='hard'
)

# 그리드 서치를 위한 하이퍼파라미터 그리드 정의
param_grid = {
    'cat__iterations': [1759],
    'cat__learning_rate': [0.8948182120285428],  # CatBoostClassifier의 학습률
    #'cat__n_estimators': [2000],  # CatBoostClassifier의 트리 개수
    'cat__depth': [7],  # 
    'cat__l2_leaf_reg': [7],
    'cat__random_state':[72],
    'cat__random_strength' :  [9.992204520122275e-05],
    'lgbm__learning_rate': [0.05803173560581885],  # LGBMClassifier의 학습률
    'lgbm__n_estimators': [742],  # LGBMClassifier의 트리 개수
    'lgbm__max_depth': [6],  # LGBMClassifier의 최대 깊이
    'lgbm__feature_fraction':[0.8501371667493939],
    'xgb__learning_rate': [0.1],  # XGBClassifier의 학습률
    'xgb__n_estimators': [2000],  # XGBClassifier의 트리 개수
    'xgb__max_depth': [4],  # XGBClassifier의 최대 깊이
}

grid_model = GridSearchCV(estimator=voting_model, param_grid=param_grid, cv=kfold)
grid_model.fit(x_train, y_train)

# 최적의 하이퍼파라미터와 결과 출력
print("최적의 하이퍼파라미터:", grid_model.best_params_)
print("최고의 평균 정확도:", grid_model.best_score_)

# 최적의 모델로 예측
y_pred = grid_model.predict(x_test)

# 평가

result = grid_model.score(x_test, y_test)
print('결과 acc:', result)
end_time = time.time() - start_time
print('걸린 시간 :' , end_time)



#===========================

# param_grid = {
#     'cat__learning_rate': [0.1],  # CatBoostClassifier의 학습률
#     'cat__n_estimators': [500],  # CatBoostClassifier의 트리 개수
#     'cat__max_depth': [4],  # CatBoostClassifier의 최대 깊이
#     'lgbm__learning_rate': [0.1],  # LGBMClassifier의 학습률
#     'lgbm__n_estimators': [500],  # LGBMClassifier의 트리 개수
#     'lgbm__max_depth': [4],  # LGBMClassifier의 최대 깊이
#     'xgb__learning_rate': [0.1],  # XGBClassifier의 학습률
#     'xgb__n_estimators': [500],  # XGBClassifier의 트리 개수
#     'xgb__max_depth': [4],  # XGBClassifier의 최대 깊이
# }
# 결과 acc: 0.9210550126673905

# ============================
# param_grid = {
#     'cat__learning_rate': [0.1],  # CatBoostClassifier의 학습률
#     'cat__n_estimators': [1000],  # CatBoostClassifier의 트리 개수
#     'cat__max_depth': [4],  # CatBoostClassifier의 최대 깊이
#     'lgbm__learning_rate': [0.1],  # LGBMClassifier의 학습률
#     'lgbm__n_estimators': [1000],  # LGBMClassifier의 트리 개수
#     'lgbm__max_depth': [4],  # LGBMClassifier의 최대 깊이
#     'xgb__learning_rate': [0.1],  # XGBClassifier의 학습률
#     'xgb__n_estimators': [1000],  # XGBClassifier의 트리 개수
#     'xgb__max_depth': [4],  # XGBClassifier의 최대 깊이
# }

# 최고의 평균 정확도: 0.9208504983400605
# 결과 acc: 0.9211002533478103

# 최고의 평균 정확도: 0.9209409922376166
# 결과 acc: 0.9212812160694897


# 최고의 평균 정확도: 0.9244641746310016
# 결과 acc: 0.9218241042345277


# 최고의 평균 정확도: 0.9259005824803485
# 결과 acc: 0.9246064060803475
    # ===================
    #     'cat__learning_rate': [0.2],  # CatBoostClassifier의 학습률
    # 'cat__n_estimators': [1000],  # CatBoostClassifier의 트리 개수
    # 'cat__max_depth': [4],  # CatBoostClassifier의 최대 깊이
    # 'lgbm__learning_rate': [0.2],  # LGBMClassifier의 학습률
    # 'lgbm__n_estimators': [1000],  # LGBMClassifier의 트리 개수
    # 'lgbm__max_depth': [4],  # LGBMClassifier의 최대 깊이
    # 'xgb__learning_rate': [0.2],  # XGBClassifier의 학습률
    # 'xgb__n_estimators': [1000],  # XGBClassifier의 트리 개수
    # 'xgb__max_depth': [4],  # XGBClassifier의 최대 깊이


#     최고의 평균 정확도: 0.9204772501890123
# 결과 acc: 0.920647846543612


#================================================
# param_grid = {
#     'cat__learning_rate': [0.09],  # CatBoostClassifier의 학습률
#     'cat__n_estimators': [1000],  # CatBoostClassifier의 트리 개수
#     'cat__max_depth': [4],  # CatBoostClassifier의 최대 깊이
#     'lgbm__learning_rate': [0.09],  # LGBMClassifier의 학습률
#     'lgbm__n_estimators': [1000],  # LGBMClassifier의 트리 개수
#     'lgbm__max_depth': [4],  # LGBMClassifier의 최대 깊이
#     'xgb__learning_rate': [0.09],  # XGBClassifier의 학습률
#     'xgb__n_estimators': [1000],  # XGBClassifier의 트리 개수
#     'xgb__max_depth': [4],  # XGBClassifier의 최대 깊이
# }


# 최고의 평균 정확도: 0.9209409922376166
# 결과 acc: 0.9212812160694897


# =====================================

# param_grid = {
#     'cat__learning_rate': [0.09],  # CatBoostClassifier의 학습률
#     'cat__n_estimators': [1500],  # CatBoostClassifier의 트리 개수
#     'cat__max_depth': [4],  # CatBoostClassifier의 최대 깊이
#     'lgbm__learning_rate': [0.09],  # LGBMClassifier의 학습률
#     'lgbm__n_estimators': [1500],  # LGBMClassifier의 트리 개수
#     'lgbm__max_depth': [4],  # LGBMClassifier의 최대 깊이
#     'xgb__learning_rate': [0.09],  # XGBClassifier의 학습률
#     'xgb__n_estimators': [1500],  # XGBClassifier의 트리 개수
#     'xgb__max_depth': [4],  # XGBClassifier의 최대 깊이
# }


#   1200
# 최고의 평균 정확도: 0.9266074761069953
# 결과 acc: 0.9251945349258053

#   1500
# 최고의 평균 정확도: 0.9276112650568343
# 결과 acc: 0.9267440282301845